# Tensorflow快速入门二
## 知识点总结
* 梯度下降方法总结
* 模型保存和恢复
# 梯度下降方法总结
* Tensorflow直接计算，满足公式$\theta=\theta-\eta \nabla_\theta M S E(\theta)$ 
* tf.random_uniform()：在图中创建一个包含随机值的节点，类似于NumPy中的random()函数
* tf.assign()：创建一个将新值赋给变量的一个节点，为variable更新值
---


In [23]:
#--coding--:utf-8
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# 获取数据
housing = fetch_california_housing()
m,n = housing.data.shape
print(housing.keys())
print(housing.feature_names)
print(housing.target)
print(housing.DESCR)

dict_keys(['data', 'target', 'feature_names', 'DESCR'])
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house val

---

# 关于警告
有点警告，但是不要紧，因为我也不确定具体的版本是多少，只能大概接近。

关于代码：
```python
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
scaled_data = scaler.fit_transform(housing.data)
data = np.c_[np.ones((m,1)),scaled_data]
```
文心一言的解释：


---


In [24]:
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
scaled_data = scaler.fit_transform(housing.data)
data = np.c_[np.ones((m,1)),scaled_data]
data

array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [25]:
# 设置参数
n_epoch = 1000
learning_rate = 0.01

# 设置placeholder
X = tf.constant(data,dtype = tf.float32,name = "X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')

theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1),name='theta')
y_pred = tf.matmul(X,theta,name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse') #Mean Squared Error 
"""均方误差"""

# 计算梯度公式
"""
麻烦在梯度公式需要我们自己手动去写，
现在只有一层，之后说不定有几千层，所以我们肯定不能自己手写，而且还不一定准确。
"""
gradient = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta,theta - learning_rate * gradient)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print("Epoch",epoch, "MSE =", mse.eval())

    print('best theta:',theta.eval())

Epoch 0 MSE = 15.371014
Epoch 100 MSE = 15.371014
Epoch 200 MSE = 15.371014
Epoch 300 MSE = 15.371014
Epoch 400 MSE = 15.371014
Epoch 500 MSE = 15.371014
Epoch 600 MSE = 15.371014
Epoch 700 MSE = 15.371014
Epoch 800 MSE = 15.371014
Epoch 900 MSE = 15.371014
best theta: [[-0.9993887 ]
 [-0.37894058]
 [-0.79117846]
 [ 0.42145848]
 [-0.7427218 ]
 [ 0.7037499 ]
 [ 0.63889766]
 [-0.92098784]
 [ 0.48915935]]


---
# 自动求导autodiff 
手动求导方法有两个缺陷：
* 深度神经网络中公式较长
* 计算效率较低 可以采取以下命令进行自动求导：
gradients = tf.gradients(mse,[theta])[0]
gradient有两个参数：
* op，损失函数，这里是mse
* variable lists，变量列表，这里是theta值
**这就已经很方便了，但还有更方便的方法**

---
# 使用Optimizer
可以使用各种优化器,如梯度下降优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
还有很多其他优化器，例如收敛更快的MomentumOptimizer优化器等
# 梯度下降中传输数据的方式
* mini-batchbatch
* 方法：使用占位符placeholder

---

In [26]:
# 传统方式
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
with tf.Session() as sess:
    test_b_1 = B.eval(feed_dict={A:[[1,2,3]]})
    test_b_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
print(test_b_1)  
print(test_b_2) 

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [27]:
# 使用mini-batch方式
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# 与上文例子相同，这里采用的是使用optimizer计算梯度
n_epochs = 100
display_step = 10
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") #X0
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


# 定义mini-batch取数据方式
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(m, size=batch_size)  
    X_batch = data[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

# mini-batch计算过程
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):#迭代的次数
        avg_cost = 0.
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            if epoch % 10 == 0:
                total_loss = 0
                acc_train = mse.eval(feed_dict={X: X_batch, y: y_batch})
                total_loss += acc_train 
                #print acc_train,total_loss
                print("epoch:",epoch, "batch_id:",batch_index, "Batch loss:", acc_train)

epoch: 0 batch_id: 0 Batch loss: 2.1672046
epoch: 0 batch_id: 1 Batch loss: 2.295659
epoch: 0 batch_id: 2 Batch loss: 1.874109
epoch: 0 batch_id: 3 Batch loss: 2.2121804
epoch: 0 batch_id: 4 Batch loss: 2.167641
epoch: 0 batch_id: 5 Batch loss: 1.7245495
epoch: 0 batch_id: 6 Batch loss: 2.0318623
epoch: 0 batch_id: 7 Batch loss: 1.5136069
epoch: 0 batch_id: 8 Batch loss: 1.6046904
epoch: 0 batch_id: 9 Batch loss: 1.3764426
epoch: 0 batch_id: 10 Batch loss: 1.3385471
epoch: 0 batch_id: 11 Batch loss: 1.7322328
epoch: 0 batch_id: 12 Batch loss: 1.4998823
epoch: 0 batch_id: 13 Batch loss: 1.9056491
epoch: 0 batch_id: 14 Batch loss: 3.0762713
epoch: 0 batch_id: 15 Batch loss: 1.2916967
epoch: 0 batch_id: 16 Batch loss: 1.5571638
epoch: 0 batch_id: 17 Batch loss: 1.598186
epoch: 0 batch_id: 18 Batch loss: 1.3730277
epoch: 0 batch_id: 19 Batch loss: 1.1644965
epoch: 0 batch_id: 20 Batch loss: 1.1785976
epoch: 0 batch_id: 21 Batch loss: 1.576241
epoch: 0 batch_id: 22 Batch loss: 1.4259119
epo

---
# 模型的保存和恢复
* 在创建图阶段创建一个Saver的节点，在执行阶段需要保存模型的地方调用Save()函数即可
* saver = tf.train.Saver()
* save_path = saver.save(sess, "model/model.ckpt")

In [30]:
# 模型的保存
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())  # 保存运行过程
            save_path = saver.save(sess, "/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/my_model_final.ckpt")#保存最后的结果
    
# 模型的加载
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() 
    
saver = tf.train.Saver({"weights":theta})


reset_graph()
saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

InvalidArgumentError: You must feed a value for placeholder tensor 'X_8' with dtype float and shape [?,9]
	 [[node X_8 (defined at C:\Users\huanyizhiyuan\AppData\Local\Temp\ipykernel_32524\966860098.py:2) ]]

Original stack trace for 'X_8':
  File "D:\Anaconda\envs\tensor_test_1x\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\envs\tensor_test_1x\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\envs\tensor_test_1x\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\envs\tensor_test_1x\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "D:\Anaconda\envs\tensor_test_1x\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "D:\Anaconda\envs\tensor_test_1x\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()